In [59]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pandas as pd
from datetime import date
import configparser


In [108]:
df = pd.read_csv("current.csv")
df = df[df['Amount'] > 0]

In [126]:


config = configparser.ConfigParser()
config.read('mappings.ini')
def diff_dates(date1, date2):
    return abs(date2-date1).days

#example on how to use diff dates
#d1 = date(2020,3,21)
#d2 = date(2020,4,5)
#result1 = diff_dates(d2, d1)

business_name_map = {'Amazon': 'Amazon', 
                     'CHERRY STREET': 'CHERRY STREET', 
                     'Grubhub': 'Grubhub', 
                     'Apple': 'Apple',
                     'Google': 'Google',
                     'Shell': 'Shell',
                     'TST*': 'TST',
                     'Coffee': 'Coffee',
                     'WHOLE FOODS': 'WHOLE FOODS MARKET',
                     'Gas': 'Gas',
                     'NYTimes': 'NYTimes',
                     'INSTACART': 'INSTACART',
                     'CVS' : 'CVS'
                     }

budget_dict = {'Bar': 500, 'Entertainment': 100, 'Grocery': 1000, 'Misc': 100, 'Pet': 100, 'Restaurant': 500, 'Shopping': 250, 'Transportation': 125}

def generate_budget():

        df_budget = pd.DataFrame.from_dict(budget_dict, orient='index', columns=['Amount'])
        df_budget.index.name= 'Category'
        
        return df_budget

def clean_name(mapping_file = business_name_map, column_name = 'Description'):

    for name in business_name_map:

        df.loc[df[column_name].str.contains(name, case=False), 'Description'] = business_name_map[name]
    
    return df

def apply_category(df, business_type):
    
    df['Category'] = df['Description'].map(business_type)
    
    df.fillna('Misc', inplace=  True)
    
    return df

def business_type(z):

    z = z.split("\n")

    cat_dict = {}

    
    for x in z:
        result = x.split(":")

        result[0] = result[0].replace("{", "")
        result[0] = result[0].replace("}", "")

        cat_dict[result[0]] = result[-1][:-1].strip()

    return cat_dict

def budget_summary(df):

    df = df[df['Amount'] > 0] #remove refunds etc
    df = clean_name(mapping_file = business_name_map, column_name = 'Description')

    df.sort_values(by = 'Amount', ascending = False).to_clipboard()

    df = apply_category(df, business_type(config['mappings']['categories']))

    df = df.groupby(by = 'Category').sum()

    df_budget = generate_budget()

    df['Budgeted'] = generate_budget()['Amount']

    df['Amount %'] = (df['Amount']/df['Budgeted'].sum())*100
    df['Budgeted %'] = (df['Budgeted']/df['Budgeted'].sum())*100
    df['Delta'] = df['Budgeted'] - df['Amount']

    df.loc['Total'] = df.sum()


    
    return df[['Amount', 'Budgeted', 'Delta', 'Amount %', 'Budgeted %']]

def transactions_by_cat(df, category):

    return df[df['Category'] == category].sort_values(by = 'Amount', ascending = False)

def find_double_charges(df):
    df.sort_values("Amount", inplace = True, ascending = True) 
    
    return df[df.duplicated(['Amount'], keep=False)].sort_values(by = 'Amount', ascending = False)

  

# Spend Summary

In [127]:

budget_summary(df)

,Amount,Budgeted,Delta,Amount %,Budgeted %
Category,,,,,
Bar,278.33,500.0,221.67,10.404860,18.691589
Entertainment,130.70,100.0,-30.70,4.885981,3.738318
Grocery,930.65,1000.0,69.35,34.790654,37.383178
Misc,69.57,100.0,30.43,2.600748,3.738318
Pet,66.02,100.0,33.98,2.468037,3.738318
Restaurant,285.45,500.0,214.55,10.671028,18.691589
Shopping,243.53,250.0,6.47,9.103925,9.345794
Transportation,873.07,125.0,-748.07,32.638131,4.672897
Total,2877.32,2675.0,-202.32,107.563364,100.000000


# Spend  by category

In [128]:
interact(transactions_by_cat, df = fixed(df), category=['Restaurant','Bar', 'Entertainment', 'Grocery', 'Misc', 'Pet', 'Shopping', 'Transportation']);

interactive(children=(Dropdown(description='category', options=('Restaurant', 'Bar', 'Entertainment', 'Grocery…

# Spend Per Vendor

In [129]:
def spend_per_vendor(filter_num):

    return (df.groupby(by = ['Description', 'Category']).sum().sort_values(by='Amount', ascending = False)).head(filter_num)

interact(spend_per_vendor, df=fixed(df), filter_num = 10)

interactive(children=(IntSlider(value=10, description='filter_num', max=30, min=-10), Output()), _dom_classes=…

<function __main__.spend_per_vendor(filter_num)>

# Average Spend by Category

In [ ]:
z = (df.groupby('Category').mean()).merge(df.groupby('Category').count(), left_index = True, right_index = True)
z['Total Spend'] = z['Amount_x'] * z['Date']
z.rename(columns = {'Date':'Count', 'Amount_x': 'Avg Amount'}, inplace = True)

z[['Count', 'Avg Amount', 'Total Spend']]

# Potential Double Charges

In [130]:
find_double_charges(df)

,Date,Description,Card Member,Account #,Amount,Category
35,4/1/20,Google,ROBERT R FUTYMA,-74003,55.04,Entertainment
36,4/1/20,Google,ROBERT R FUTYMA,-74003,55.04,Entertainment
30,4/2/20,CLOUDBURST BREWING,ROBERT R FUTYMA,-74003,43.00,Bar
61,3/24/20,CLOUDBURST BREWING,ROBERT R FUTYMA,-74003,43.00,Bar
0,4/11/20,PRIME VIDEO*YL7B925T888-802-3080 WA,ROBERT R FUTYMA,-74003,16.50,Misc
10,4/7/20,ADOBE ACROPRO SUBS ASAN JOSE CA,ROBERT R FUTYMA,-74003,16.50,Misc
39,3/31/20,SDOT PARKING PAYBYPHONE,ROBERT R FUTYMA,-74003,1.50,Transportation
59,3/26/20,SDOT PARKING PAYBYPHONE,ROBERT R FUTYMA,-74003,1.50,Transportation
37,4/1/20,SDOT PARKING PAYBYPHONE,ROBERT R FUTYMA,-74003,1.50,Transportation
83,3/21/20,SDOT PARKING PAYBYPHONE,ROBERT R FUTYMA,-74003,1.50,Transportation
